In [1]:
import deepemogp.feature_extractor as feature_extractor
import deepemogp.signal.physio as physio
import deepemogp.signal.face as face
import deepemogp.signal.annotation as annotation
import deepemogp.datasets as datasets

# from: https://github.com/SheffieldML/GPy
#import GPy
# from: https://github.com/SheffieldML/PyDeepGP
#import deepgp

import pandas as pd
import numpy as np


In [2]:
show = True
loadData = True
learnModel = True

In [3]:

#if loadData:

# definition of the feature extractors to be used later
f1 = feature_extractor.FE('wavelet')
f2 = feature_extractor.FE('wavelet')
f3 = feature_extractor.FE('mean')

In [4]:

# definition of the physiological signals to be extracted
skt = physio.SKT(f1)
hr  = physio.HR(f2)
#of  = face.FP_OF(f3)

# definition of the emotional annotation to be extracted
#va = annotation.VA('valence', f3)
#ar = annotation.VA('arousal', f3)

In [5]:
# extraction of the desired data from the dataset
d = datasets.FEAR(signals={hr}, subjects={'1_1'})
print(d)

>> Loading HR for subject 1 and session 1 from dataset FEAR
[{'data': [0.0273132, 0.0341797, 0.0411987, 0.0489807, 0.0556946, 0.0617981, 0.0675964, 0.0727844, 0.0779724, 0.0827026, 0.0871277, 0.0912476, 0.0949097, 0.0987244, 0.102692, 0.106201, 0.109558, 0.112152, 0.114594, 0.115662, 0.115204, 0.112762, 0.10849, 0.101166, 0.0921631, 0.0814819, 0.0688171, 0.0540161, 0.0382996, 0.0204468, 0.00274658, -0.0163269, -0.0341797, -0.050354, -0.0662231, -0.0808716, -0.0950623, -0.10788, -0.118256, -0.126801, -0.132599, -0.136719, -0.139923, -0.142822, -0.144806, -0.1474, -0.149231, -0.150146, -0.151215, -0.151825, -0.151367, -0.150757, -0.149231, -0.146484, -0.14328, -0.140533, -0.138397, -0.136566, -0.135651, -0.135651, -0.136261, -0.137482, -0.13855, -0.140076, -0.141144, -0.142059, -0.14328, -0.144043, -0.14267, -0.137634, -0.126038, -0.104065, -0.0694275, -0.0213623, 0.040741, 0.116272, 0.2034, 0.301514, 0.408325, 0.52002, 0.633698, 0.745544, 0.850525, 0.946045, 1.02905, 1.09543, 1.13968, 1

In [6]:
data = np.load('../fear_gen/prova.npy', allow_pickle=True).item()
eda = data['eda']

FileNotFoundError: [Errno 2] No such file or directory: '../fear_gen/prova.npy'

In [ ]:
signal_ = eda
signal_.feature_ext.extract_feat(signal_, show=show)

In [ ]:
for s in d.signals:
	# preprocess ...
	s.preprocess(show=show)

	# ... and extract features from each signal type
	s.feature_ext.extract_feat(s, show=show)

	print(s)

for a in d.annotations:
	# preprocess ...
	a.preprocess(ewe=False, show=show)

	# ... and extract features for each annotation type
	a.feature_ext.extract_feat(a, show=show)
	print(a)


# Saving a model
# np.save('data.npy', d)

'''
else:

	d = np.load('data.npy') # Load the data
	d = d.item()
'''

In [ ]:

# create model
selected_view = 'HR'

# observed inputs variables (annotations)
X = pd.concat([a.features for a in d.annotations], axis=1).values
# observed output variables (measured signals)
Y = [s.features.values for s in d.signals if s.name==selected_view][0]

Xmean = X.mean()
Xstd = X.std()

#X-=Xmean
#X/=Xstd

N = len(X)

# Number of latent dimensions (single hidden layer, since the top layer is observed)
Q = [Y.shape[1], 6, X.shape[1]]

k1 = GPy.kern.RBF(Q[1], ARD=True) + GPy.kern.Bias(Q[1]) + GPy.kern.White(Q[1])

k2 = GPy.kern.Linear(Q[2], ARD=False)

# Dimensions of the MLP back-constraint if set to true
encoder_dims=[[300],[150]]

#inits = ['PPCA', 'PPCA' , X]

# deepGP
#m = deepgp.DeepGP([d1, Q1, X.shape[1]], Y=Ytr, X=Xtr,
m = deepgp.DeepGP(Q, Y=Y, X=X,
	kernels=[k1, k2],
	num_inducing=N,
	back_constraint=True,
	encoder_dims=encoder_dims)


if learnModel:
	#--------- Optimization ----------#
	for i in range(len(m.layers)):
		# Make sure initial noise variance gives a reasonable signal to noise ratio.
		# Fix to that value for a few iterations to avoid early local minima
		output_var = m.layers[i].Y.var() if i==0 else m.layers[i].Y.mean.var()
		m.layers[i].Gaussian_noise.variance = output_var *  0.1
		m.layers[i].Gaussian_noise.variance.fix()

	m.optimize(max_iters=1000, messages=1)
	deepgp.util.check_snr(m)

	# Now unfix noise and learn normally.
	for i in range(len(m.layers)):
		m.layers[i].Gaussian_noise.variance.unfix()

	#m.optimize_restarts(parallel=False, messages=1, robust=True, num_restarts=3, max_iters=250)
	m.optimize(max_iters=10000, messages=1)
	deepgp.util.check_snr(m)

	print(m)

	# Saving a model
	np.save('model_save.npy', m.param_array)

else:

	m.update_model(False) # do not call the underlying expensive algebra on load
	m.initialize_parameter() # Initialize the parameters (connect the parameters up)
	m[:] = np.load('model_save.npy') # Load the parameters
	m.update_model(True) # Call the algebra only once


# Show ARD results
import matplotlib.pyplot as plt
m.obslayer.kern.plot_ARD('rbf')
plt.show()


# Generate session - Experiment 1
from deepemogp.signal.utils import utils
utils.generate_session(m, [selected_view], d)









# pred = m.predict(Xts)[0]

# rmse = np.sqrt(((pred.flatten() - Yts.flatten()) ** 2).mean())

# print "prediction RMSE: %f" % (rmse)


# import matplotlib.pyplot as plt
# f = plt.figure(); plt.plot(pred.T, 'x-');  plt.plot(Yts.T, 'o'); f.show();